# Simple Backtest Example

This notebook demonstrates a fast backtest of portfolio optimization algorithms during a volatile period where A2A ensemble methods can shine.

## 1. Import Libraries

Import the necessary libraries for backtesting.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# AlloOptim imports
from allooptim.backtest.backtest_config import BacktestConfig
from allooptim.backtest.backtest_engine import BacktestEngine

print("Libraries imported successfully!")

INFO:allooptim:AlloOptim package initialized successfully


Libraries imported successfully!


## 2. Configure Backtest

Set up a fast backtest for a volatile period (COVID crash) where A2A ensemble methods excel.

In [2]:
# Configure a fast backtest for volatile period (COVID crash)
config = BacktestConfig(
    # Required fields
    start_date=datetime(2020, 3, 1),
    end_date=datetime(2020, 6, 30),
    # Short volatile period where A2A shines (March-June 2020)
    quick_start_date=datetime(2020, 3, 1),
    quick_end_date=datetime(2020, 6, 30),
    # Fast settings for quick execution
    rebalance_frequency=5,  # Rebalance every 5 trading days
    lookback_days=30,  # Use 30 days of historical data
    # Simple asset universe
    symbols=["AAPL", "MSFT", "AMZN", "GOOGL", "TSLA", "SPY"],
    # Use A2A with a few diverse optimizers
    optimizer_names=["MaxSharpe", "RiskParityOptimizer", "NaiveOptimizer", "MomentumOptimizer", "HRPOptimizer", "NCOSharpeOptimizer"],
    # Quick test mode
    quick_test=True,
    # Do not store results to disk for notebook environment
    store_results=False,
)

## 3. Run Backtest

Execute the fast backtest to see how individual optimizers and A2A ensemble perform during market volatility.

In [3]:
# Initialize and run the backtest
print("Initializing backtest engine...")
backtest_engine = BacktestEngine(config)

print("Running fast backtest...")
print("This should complete in under 30 seconds...")

# Run the backtest
results = backtest_engine.run_backtest()

if results:
    print("✅ Backtest completed successfully!")
    print(f"   Tested {len([k for k in results.keys() if k != 'SPY_Benchmark'])} optimizers + A2A + benchmark")
else:
    print("❌ Backtest failed")
    raise RuntimeError("Backtest execution failed")

2025-11-06 06:19:07,309 - allooptim.backtest.data_loader - INFO - Loaded universe with 6 symbols
2025-11-06 06:19:07,313 - allooptim.backtest.backtest_engine - INFO - Starting comprehensive backtest
2025-11-06 06:19:07,313 - allooptim.backtest.backtest_engine - INFO - Starting comprehensive backtest
2025-11-06 06:19:07,313 - allooptim.backtest.data_loader - INFO - Loading price data from 2020-01-31 00:00:00 to 2020-06-30 00:00:00
2025-11-06 06:19:07,320 - allooptim.backtest.data_loader - INFO - Loading batch 1: 6 symbols
2025-11-06 06:19:07,313 - allooptim.backtest.data_loader - INFO - Loading price data from 2020-01-31 00:00:00 to 2020-06-30 00:00:00
2025-11-06 06:19:07,320 - allooptim.backtest.data_loader - INFO - Loading batch 1: 6 symbols


Initializing backtest engine...
Running fast backtest...
This should complete in under 30 seconds...


2025-11-06 06:19:09,223 - allooptim.backtest.data_cleaning - INFO - Raw price data shape: (104, 6)
2025-11-06 06:19:09,229 - allooptim.backtest.data_cleaning - WARNING - Very few symbols remain after cleaning: 6 (required: 10)
2025-11-06 06:19:09,233 - allooptim.backtest.data_cleaning - INFO - Clean price data shape: (104, 6) (dropped 0 symbols)
2025-11-06 06:19:09,229 - allooptim.backtest.data_cleaning - WARNING - Very few symbols remain after cleaning: 6 (required: 10)
2025-11-06 06:19:09,233 - allooptim.backtest.data_cleaning - INFO - Clean price data shape: (104, 6) (dropped 0 symbols)
2025-11-06 06:19:09,235 - allooptim.backtest.data_loader - INFO - Loaded data for 6 assets over 104 days
2025-11-06 06:19:09,240 - allooptim.backtest.backtest_engine - INFO - Running backtest with 15 rebalancing dates
2025-11-06 06:19:09,240 - allooptim.backtest.backtest_engine - INFO - Processing rebalance 1/15: 2020-03-16 00:00:00
2025-11-06 06:19:09,235 - allooptim.backtest.data_loader - INFO - Lo

✅ Backtest completed successfully!
   Tested 8 optimizers + A2A + benchmark


## 4. Analyze Results

Compare the performance of individual optimizers vs A2A ensemble during the volatile COVID period.

In [4]:
# Extract performance data for analysis
performance_data = []

for name, data in results.items():
    if 'metrics' in data and 'sharpe_ratio' in data['metrics']:
        row = {'strategy': name}
        row.update(data['metrics'])
        performance_data.append(row)

# Create DataFrame for analysis
df_results = pd.DataFrame(performance_data)
df_results = df_results.sort_values('sharpe_ratio', ascending=False)

print("Backtest Results Summary (COVID Crash Period):")
print("=" * 60)
print(f"Test period: March-June 2020 (High volatility)")
print(f"Strategies tested: {len(df_results)}")
print(f"Available strategies: {', '.join(df_results['strategy'].tolist())}")
print()

# Show top performers
print("Performance Rankings:")
print("-" * 40)
for i, row in df_results.head().iterrows():
    print(f"{i+1:2d}. {row['strategy']}: Sharpe = {row['sharpe_ratio']:.3f}")

print()

# Check if A2A outperformed individual optimizers
a2a_sharpe = df_results[df_results['strategy'] == 'A2AEnsemble']['sharpe_ratio'].iloc[0]
individual_avg = df_results[df_results['strategy'] != 'A2AEnsemble']['sharpe_ratio'].mean()

print("A2A Ensemble Analysis:")
print("-" * 40)
print(f"A2A Sharpe Ratio: {a2a_sharpe:.3f}")
print(f"Average Individual Sharpe: {individual_avg:.3f}")

# Check for benchmark if available
benchmark_strategies = [s for s in df_results['strategy'] if 'benchmark' in s.lower() or 'spy' in s.lower()]
if benchmark_strategies:
    spy_sharpe = df_results[df_results['strategy'] == benchmark_strategies[0]]['sharpe_ratio'].iloc[0]
    print(f"Benchmark ({benchmark_strategies[0]}) Sharpe: {spy_sharpe:.3f}")
    print(f"A2A vs Benchmark: {a2a_sharpe - spy_sharpe:.3f}")
else:
    print("No benchmark strategy found in results")

print(f"A2A vs Individual: {a2a_sharpe - individual_avg:.3f}")
print()

Backtest Results Summary (COVID Crash Period):
Test period: March-June 2020 (High volatility)
Strategies tested: 8
Available strategies: MaxSharpe, MomentumOptimizer, HRPOptimizer, RiskParityOptimizer, NaiveOptimizer, A2AEnsemble, NCOSharpeOptimizer, SPYBenchmark

Performance Rankings:
----------------------------------------
 1. MaxSharpe: Sharpe = 3.240
 4. MomentumOptimizer: Sharpe = 3.101
 6. HRPOptimizer: Sharpe = 2.101
 2. RiskParityOptimizer: Sharpe = 2.092
 3. NaiveOptimizer: Sharpe = 2.092

A2A Ensemble Analysis:
----------------------------------------
A2A Sharpe Ratio: 2.043
Average Individual Sharpe: 2.187
Benchmark (SPYBenchmark) Sharpe: 0.995
A2A vs Benchmark: 1.048
A2A vs Individual: -0.144

